<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Build Fast with AI](https://img.shields.io/badge/BuildFastWithAI-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://www.buildfastwithai.com/genai-course)
[![EduChain GitHub](https://img.shields.io/github/stars/satvik314/educhain?style=for-the-badge&logo=github&color=gold)](https://github.com/satvik314/educhain)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1iLK4oqazJ1hU64qb3TMO9_GCbJ7cDzmS?usp=sharing)
## Master Generative AI in 6 Weeks
**What You'll Learn:**
- Build with Latest LLMs
- Create Custom AI Apps
- Learn from Industry Experts
- Join Innovation Community
Transform your AI ideas into reality through hands-on projects and expert mentorship.
[Start Your Journey](https://www.buildfastwithai.com/genai-course)
*Empowering the Next Generation of AI Innovators

# 📌 **Neon_DB_Serverless_PGVector**
**Neon** is an **open-source, serverless PostgreSQL database** with features like **autoscaling, branching, and vector support (`pgvector`)**. It is ideal for **AI applications, real-time analytics, and scalable cloud databases**.

## 🚀 **Key Features**
✅ **Open Source** - Fully transparent and free to use  
✅ **Serverless** - No need to manage infrastructure  
✅ **Autoscaling** - Automatically adjusts resources  
✅ **Branching** - Instant database cloning for testing  
✅ **Vector Search (`pgvector`)** - Ideal for AI and embeddings  
✅ **Compatible with PostgreSQL** - Works with all PostgreSQL tools  

### **🔌 Connect to Neon PostgreSQL**

In [ ]:
import os
import psycopg2
from google.colab import userdata

connection_string = userdata.get('NEON_URL')

connection = psycopg2.connect(connection_string)

cursor = connection.cursor()

### **✅ Test Neon PostgreSQL Connection**








In [ ]:
cursor.execute("SELECT 1;")
result = cursor.fetchone()

if result == (1,):
    print("Your database connection was successful!")
else:
    print("Your connection failed.")

Your database connection was successful!


### **📦 Install pgvector Extension in Neon DB**








In [ ]:
cursor.execute("CREATE EXTENSION IF NOT EXISTS vector;")


### **🗄️ Create Table & Insert Vector Data**








In [ ]:
create_table_sql = '''
CREATE TABLE items (
  id BIGSERIAL PRIMARY KEY,
  embedding VECTOR(3)
);
'''

insert_data_sql = '''
INSERT INTO items (embedding) VALUES ('[1,2,3]'), ('[4,5,6]'), ('[7,8,9]');
'''

cursor.execute(create_table_sql)
cursor.execute(insert_data_sql)

connection.commit()

### **🔍 Perform Vector Similarity Search**



In [ ]:
cursor.execute("SELECT * FROM items ORDER BY embedding <-> '[3,1,2]' LIMIT 3;")
all_data = cursor.fetchall()
print(all_data)

[(1, '[1,2,3]'), (2, '[4,5,6]'), (3, '[7,8,9]')]


###**Interactive To-Do List App With Neon DB 📝**

In [ ]:
conn = psycopg2.connect(connection_string)
cursor = conn.cursor()

### **1️⃣ Setup Database 🗄️**  


In [ ]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS tasks (
        id SERIAL PRIMARY KEY,
        task TEXT NOT NULL,
        completed BOOLEAN DEFAULT FALSE
    )
""")
conn.commit()

### **2️⃣ Add a New Task ➕**  


In [ ]:
def add_task(task):
    cursor.execute("INSERT INTO tasks (task) VALUES (%s)", (task,))
    conn.commit()
    print("✅ Task added!")


### **3️⃣ View All Tasks 📜**  


In [ ]:
def view_tasks():
    cursor.execute("SELECT id, task, completed FROM tasks")
    tasks = cursor.fetchall()
    print("\n📋 To-Do List:")
    for task in tasks:
        status = "✅" if task[2] else "❌"
        print(f"{task[0]}. {task[1]} {status}")


### **4️⃣ Delete a Task ❌**  


In [ ]:
def delete_task(task_id):
    cursor.execute("DELETE FROM tasks WHERE id = %s", (task_id,))
    conn.commit()
    print("🗑️ Task deleted!")


In [ ]:
while True:
    print("\n1. Add Task  2. View Tasks  3. Delete Task  4. Exit")
    choice = input("Choose an option: ")

    if choice == "1":
        task = input("Enter task: ")
        add_task(task)
    elif choice == "2":
        view_tasks()
    elif choice == "3":
        task_id = input("Enter task ID to delete: ")
        delete_task(task_id)
    elif choice == "4":
        break
    else:
        print("❌ Invalid choice!")
cursor.close()
conn.close()
print("✅ Database connection closed.")


1. Add Task  2. View Tasks  3. Delete Task  4. Exit
Choose an option: 1
Enter task: Meeting at 7pm
✅ Task added!

1. Add Task  2. View Tasks  3. Delete Task  4. Exit
Choose an option: 2

📋 To-Do List:
1. Meeting at 7pm ❌

1. Add Task  2. View Tasks  3. Delete Task  4. Exit
Choose an option: 3
Enter task ID to delete: 1
🗑️ Task deleted!

1. Add Task  2. View Tasks  3. Delete Task  4. Exit
Choose an option: 2

📋 To-Do List:

1. Add Task  2. View Tasks  3. Delete Task  4. Exit
Choose an option: 4
✅ Database connection closed.


# **💬 OpenAI Chatbot with NeonDB for Chat History 🗄️**


In [ ]:
from google.colab import userdata
from openai import OpenAI

OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

In [ ]:
import psycopg2
from openai import OpenAI

conn = psycopg2.connect(connection_string)
conn.autocommit = True
cursor = conn.cursor()

cursor.execute("""
    CREATE TABLE IF NOT EXISTS chat_history (
        id SERIAL PRIMARY KEY,
        user_message TEXT NOT NULL,
        bot_response TEXT NOT NULL,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
""")

client = OpenAI(api_key=OPENAI_API_KEY)

def get_chat_history(limit=5):
    """Retrieve the last 'limit' messages for context."""
    cursor.execute(f"SELECT user_message, bot_response FROM chat_history ORDER BY created_at DESC LIMIT {limit};")
    return cursor.fetchall()[::-1]

def chat_with_gpt(user_input):
    chat_history = get_chat_history(limit=5)

    messages = [{"role": "system", "content": "You are a helpful assistant with memory. Remember past user inputs."}]

    for user_msg, bot_msg in chat_history:
        messages.append({"role": "user", "content": user_msg})
        messages.append({"role": "assistant", "content": bot_msg})

    messages.append({"role": "user", "content": user_input})

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )

    bot_response = response.choices[0].message.content

    cursor.execute(
        "INSERT INTO chat_history (user_message, bot_response) VALUES (%s, %s)",
        (user_input, bot_response),
    )

    return bot_response

while True:
    user_message = input("You: ")
    if user_message.lower() in ["exit", "quit"]:
        print("Goodbye!")
        break

    bot_response = chat_with_gpt(user_message)
    print(f"GPT-4o: {bot_response}")

cursor.close()
conn.close()
print("✅ Database connection closed.")
